# Command Line und Libvirt


In [1]:
apt policy lvm2

lvm2:
  Installiert:           2.02.133-1ubuntu10
  Installationskandidat: 2.02.133-1ubuntu10
  Versionstabelle:
 *** 2.02.133-1ubuntu10 500
        500 http://us.archive.ubuntu.com/ubuntu xenial/main amd64 Packages
        100 /var/lib/dpkg/status


In [2]:
sudo fdisk -l /dev/sdb

Medium /dev/sdb: 500 GiB, 536870912000 Bytes, 1048576000 Sektoren
Einheiten: sectors von 1 * 512 = 512 Bytes
Sektorengröße (logisch/physisch): 512 Bytes / 512 Bytes
I/O Größe (minimal/optimal): 512 Bytes / 512 Bytes
Typ der Medienbezeichnung: gpt
Medienkennung: 45D6BF56-0CA8-4BEC-A655-CCB23EE16F88

Gerät      Start       Ende   Sektoren Größe Typ
/dev/sdb1   1024 1048575966 1048574943  500G Linux LVM


```bash
sudo fdisk /dev/sdb
```

1. Press n for creating a new partition.
2. Then choose p for primary partition.
3. Next, choose the partition number 1.
4. Use the default value by just pressing the Enter key two times.
5. Next, press p to print the defined partition.
6. Press l to list all available types.
7. Type t to choose the partitions.
8. Choose 8e for the Linux LVM and press Enter to apply.
9. Again use p to print the changes.
10. Finally, use w to save the changes:


In [3]:
apt policy gdisk

gdisk:
  Installiert:           1.0.1-1build1
  Installationskandidat: 1.0.1-1build1
  Versionstabelle:
 *** 1.0.1-1build1 500
        500 http://us.archive.ubuntu.com/ubuntu xenial/main amd64 Packages
        100 /var/lib/dpkg/status


In [4]:
sudo sgdisk -d 1 -a 1024 -n 1:1024: -c 1:"External HD" -t 1:8e00 /dev/sdb 

Setting name!
partNum is 0
REALLY setting name!
The operation has completed successfully.


In [5]:
sudo sgdisk -p /dev/sdb

Disk /dev/sdb: 1048576000 sectors, 500.0 GiB
Logical sector size: 512 bytes
Disk identifier (GUID): 45D6BF56-0CA8-4BEC-A655-CCB23EE16F88
Partition table holds up to 128 entries
First usable sector is 34, last usable sector is 1048575966
Partitions will be aligned on 1024-sector boundaries
Total free space is 990 sectors (495.0 KiB)

Number  Start (sector)    End (sector)  Size       Code  Name
   1            1024      1048575966   500.0 GiB   8E00  External HD


In [6]:
sudo pvcreate /dev/sdb1

  Physical volume "/dev/sdb1" successfully created


In [7]:
sudo vgcreate lxc /dev/sdb1

  Volume group "lxc" successfully created


In [8]:
sudo lxc-create --bdev lvm --fssize 10G --name lvm_container --template ubuntu

Checking cache download in /var/cache/lxc/xenial/rootfs-amd64 ... 
Copy /var/cache/lxc/xenial/rootfs-amd64 to /usr/lib/x86_64-linux-gnu/lxc ... 
Copying rootfs to /usr/lib/x86_64-linux-gnu/lxc ...
Generating locales (this might take a while)...
  de_DE.UTF-8... done
Generation complete.
debconf: kann Oberfläche nicht initialisieren: Dialog
debconf: (Die Dialog-Oberfläche funktioniert nicht auf einem Dumb-Terminal, einem Emacs-Shellbuffer oder ohne ein steuerndes Terminal.)
debconf: greife zurück auf die Oberfläche: Readline
debconf: kann Oberfläche nicht initialisieren: Readline
debconf: (Can't locate Term/ReadLine.pm in @INC (you may need to install the Term::ReadLine module) (@INC contains: /etc/perl /usr/local/lib/x86_64-linux-gnu/perl/5.22.1 /usr/local/share/perl/5.22.1 /usr/lib/x86_64-linux-gnu/perl5/5.22 /usr/share/perl5 /usr/lib/x86_64-linux-gnu/perl/5.22 /usr/share/perl/5.22 /usr/local/lib/site_perl /usr/lib/x86_64-linux-gnu/perl-base .) at /usr/share/perl5/Debconf/FrontEnd/Rea

In [9]:
sudo lxc-ls --fancy

NAME          STATE   AUTOSTART GROUPS IPV4 IPV6 
lvm_container STOPPED 0         -      -    -    


In [10]:
sudo pvs

  PV         VG         Fmt  Attr PSize   PFree  
  /dev/sda5  rdf-dev-vg lvm2 a--  126,52g      0 
  /dev/sdb1  lxc        lvm2 a--  500,00g 490,00g


In [11]:
sudo vgs

  VG         #PV #LV #SN Attr   VSize   VFree  
  lxc          1   1   0 wz--n- 500,00g 490,00g
  rdf-dev-vg   1   2   0 wz--n- 126,52g      0 


In [12]:
sudo lvs

  LV            VG         Attr       LSize   Pool Origin Data%  Meta%  Move Log Cpy%Sync Convert
  lvm_container lxc        -wi-a-----  10,00g                                                    
  root          rdf-dev-vg -wi-ao---- 124,52g                                                    
  swap_1        rdf-dev-vg -wi-ao----   2,00g                                                    


In [13]:
sudo lxc-start -n lvm_container

In [14]:
sudo lxc-ls --fancy

NAME          STATE   AUTOSTART GROUPS IPV4 IPV6 
lvm_container RUNNING 0         -      -    -    


In [15]:
sudo grep -vi ^# /var/lib/lxc/lvm_container/config | grep -v ^$

lxc.include = /usr/share/lxc/config/ubuntu.common.conf
lxc.rootfs.path = lvm:/dev/lxc/lvm_container
lxc.uts.name = lvm_container
lxc.arch = amd64
lxc.net.0.type = veth
lxc.net.0.link = lxcbr0
lxc.net.0.flags = up
lxc.net.0.hwaddr = 00:16:3e:1a:01:b6


In [16]:
ls -la /dev/lxc

insgesamt 0
drwxr-xr-x  2 root root   60 Feb 17 17:59 .
drwxr-xr-x 20 root root 4000 Feb 17 17:59 ..
lrwxrwxrwx  1 root root    7 Feb 17 17:59 lvm_container -> ../dm-2


In [17]:
ls -la /dev/mapper

insgesamt 0
drwxr-xr-x  2 root root     120 Feb 17 17:59 .
drwxr-xr-x 20 root root    4000 Feb 17 17:59 ..
crw-------  1 root root 10, 236 Feb 17 16:18 control
lrwxrwxrwx  1 root root       7 Feb 17 17:59 lxc-lvm_container -> ../dm-2
lrwxrwxrwx  1 root root       7 Feb 17 16:18 rdf--dev--vg-root -> ../dm-0
lrwxrwxrwx  1 root root       7 Feb 17 16:18 rdf--dev--vg-swap_1 -> ../dm-1


In [18]:
ls -la /dev/mapper/lxc-lvm_container

lrwxrwxrwx 1 root root 7 Feb 17 17:59 /dev/mapper/lxc-lvm_container -> ../dm-2


In [19]:
sudo lxc-create --bdev lvm --fssize 5G --name lvm_container_2 --template ubuntu

Checking cache download in /var/cache/lxc/xenial/rootfs-amd64 ... 
Copy /var/cache/lxc/xenial/rootfs-amd64 to /usr/lib/x86_64-linux-gnu/lxc ... 
Copying rootfs to /usr/lib/x86_64-linux-gnu/lxc ...
Generating locales (this might take a while)...
  de_DE.UTF-8... done
Generation complete.
debconf: kann Oberfläche nicht initialisieren: Dialog
debconf: (Die Dialog-Oberfläche funktioniert nicht auf einem Dumb-Terminal, einem Emacs-Shellbuffer oder ohne ein steuerndes Terminal.)
debconf: greife zurück auf die Oberfläche: Readline
debconf: kann Oberfläche nicht initialisieren: Readline
debconf: (Can't locate Term/ReadLine.pm in @INC (you may need to install the Term::ReadLine module) (@INC contains: /etc/perl /usr/local/lib/x86_64-linux-gnu/perl/5.22.1 /usr/local/share/perl/5.22.1 /usr/lib/x86_64-linux-gnu/perl5/5.22 /usr/share/perl5 /usr/lib/x86_64-linux-gnu/perl/5.22 /usr/share/perl/5.22 /usr/local/lib/site_perl /usr/lib/x86_64-linux-gnu/perl-base .) at /usr/share/perl5/Debconf/FrontEnd/Rea

In [20]:
sudo lvs

  LV              VG         Attr       LSize   Pool Origin Data%  Meta%  Move Log Cpy%Sync Convert
  lvm_container   lxc        -wi-ao----  10,00g                                                    
  lvm_container_2 lxc        -wi-a-----   5,00g                                                    
  root            rdf-dev-vg -wi-ao---- 124,52g                                                    
  swap_1          rdf-dev-vg -wi-ao----   2,00g                                                    


In [21]:
ls -la /dev/lxc

insgesamt 0
drwxr-xr-x  2 root root   80 Feb 17 17:59 .
drwxr-xr-x 20 root root 4020 Feb 17 17:59 ..
lrwxrwxrwx  1 root root    7 Feb 17 17:59 lvm_container -> ../dm-2
lrwxrwxrwx  1 root root    7 Feb 17 18:00 lvm_container_2 -> ../dm-3


In [22]:
sudo lvdisplay

  --- Logical volume ---
  LV Path                /dev/rdf-dev-vg/root
  LV Name                root
  VG Name                rdf-dev-vg
  LV UUID                zr58k9-rXoY-1UDX-ncQg-b4WL-6k4B-dF0P3b
  LV Write Access        read/write
  LV Creation host, time rdf-dev, 2018-02-16 15:12:14 +0000
  LV Status              available
  # open                 1
  LV Size                124,52 GiB
  Current LE             31877
  Segments               1
  Allocation             inherit
  Read ahead sectors     auto
  - currently set to     256
  Block device           252:0
   
  --- Logical volume ---
  LV Path                /dev/rdf-dev-vg/swap_1
  LV Name                swap_1
  VG Name                rdf-dev-vg
  LV UUID                1z93Fs-6BnF-V7Ot-BvIj-EHQW-AiKe-PnlafY
  LV Write Access        read/write
  LV Creation host, time rdf-dev, 2018-02-16 15:12:15 +0000
  LV Status              available
  # open                 2
  LV Size                2,00 GiB
  Current LE           

## Snapshots von Containern mit LVM

In [23]:
sudo lxc-copy --snapshot --name lvm_container_2 --newname container_2_copy

In [24]:
sudo lxc-ls --fancy

NAME             STATE   AUTOSTART GROUPS IPV4       IPV6 
container_2_copy STOPPED 0         -      -          -    
lvm_container    RUNNING 0         -      10.0.3.147 -    
lvm_container_2  STOPPED 0         -      -          -    


In [25]:
sudo lvs

  LV               VG         Attr       LSize   Pool Origin          Data%  Meta%  Move Log Cpy%Sync Convert
  container_2_copy lxc        swi-a-s---   5,00g      lvm_container_2 0,00                                   
  lvm_container    lxc        -wi-ao----  10,00g                                                             
  lvm_container_2  lxc        owi-a-s---   5,00g                                                             
  root             rdf-dev-vg -wi-ao---- 124,52g                                                             
  swap_1           rdf-dev-vg -wi-ao----   2,00g                                                             


In [26]:
sudo lxc-copy --name lvm_container_2 --newname container_2_hard

In [27]:
sudo lvs

  LV               VG         Attr       LSize   Pool Origin          Data%  Meta%  Move Log Cpy%Sync Convert
  container_2_copy lxc        swi-a-s---   5,00g      lvm_container_2 0,18                                   
  container_2_hard lxc        -wi-a-----   5,00g                                                             
  lvm_container    lxc        -wi-ao----  10,00g                                                             
  lvm_container_2  lxc        owi-a-s---   5,00g                                                             
  root             rdf-dev-vg -wi-ao---- 124,52g                                                             
  swap_1           rdf-dev-vg -wi-ao----   2,00g                                                             


## Erstellen eines Blockdevice

In [28]:
pwd

/home/vagrant/rdf/ais/hands-on-lxc


In [29]:
truncate --size 5G ../xvdz.img

In [30]:
ls -la ../xvdz.img

-rw-r--r-- 1 vagrant vagrant 5368709120 Feb 17 18:00 ../xvdz.img


In [31]:
sudo modprobe loop

In [32]:
sudo losetup --find

/dev/loop0


In [33]:
sudo losetup /dev/loop0 ../xvdz.img

In [34]:
sudo losetup --all

/dev/loop0: [64512]:2497524 (/home/vagrant/rdf/ais/xvdz.img)


In [35]:
sudo pvcreate /dev/loop0

  Physical volume "/dev/loop0" successfully created


## Alternative mit dd

In [36]:
sudo  dd if=/dev/zero of=/block_device bs=1k count=500000

500000+0 Datensätze ein
500000+0 Datensätze aus
512000000 bytes (512 MB, 488 MiB) copied, 2,01046 s, 255 MB/s


In [37]:
sudo losetup --find

/dev/loop1


In [38]:
sudo losetup /dev/loop1 /block_device

In [39]:
sudo pvcreate /dev/loop1

  Physical volume "/dev/loop1" successfully created


In [40]:
sudo losetup --all

/dev/loop0: [64512]:2497524 (/home/vagrant/rdf/ais/xvdz.img)
/dev/loop1: [64512]:16 (/block_device)


## Löschen der Blockdevices

In [41]:
sudo losetup --detach /dev/loop0

In [42]:
sudo losetup --detach /dev/loop1

In [43]:
sudo losetup --all

## Btrfs Speicher

In [44]:
apt policy btrfs-tools

btrfs-tools:
  Installiert:           4.4-1ubuntu1
  Installationskandidat: 4.4-1ubuntu1
  Versionstabelle:
 *** 4.4-1ubuntu1 500
        500 http://us.archive.ubuntu.com/ubuntu xenial-updates/main amd64 Packages
        100 /var/lib/dpkg/status
     4.4-1 500
        500 http://us.archive.ubuntu.com/ubuntu xenial/main amd64 Packages


In [45]:
sudo apt install -y btrfs-tools

Paketlisten werden gelesen... Fertig
Abhängigkeitsbaum wird aufgebaut.       
Statusinformationen werden eingelesen.... Fertig
»btrfs-tools« ist bereits die neuste Version (4.4-1ubuntu1).
0 aktualisiert, 0 neu installiert, 0 zu entfernen und 0 nicht aktualisiert.


In [46]:
lsmod | grep btrfs

btrfs                 991232  0
xor                    24576  1 btrfs
raid6_pq              102400  1 btrfs


In [47]:
sudo modprobe btrfs

In [48]:
lsmod | grep btrfs

btrfs                 991232  0
xor                    24576  1 btrfs
raid6_pq              102400  1 btrfs


In [49]:
sudo losetup --find

/dev/loop0


In [50]:
ls -la ../xvdz.img

-rw-r--r-- 1 vagrant vagrant 5368709120 Feb 17 18:00 ../xvdz.img


In [51]:
sudo losetup /dev/loop0 ../xvdz.img

In [52]:
sudo mkfs -t btrfs -f /dev/loop0

btrfs-progs v4.4
See http://btrfs.wiki.kernel.org for more information.

Performing full device TRIM (5.00GiB) ...
Label:              (null)
UUID:               f21bfaef-88ef-40ac-b5f4-045e2d84f7a2
Node size:          16384
Sector size:        4096
Filesystem size:    5.00GiB
Block group profiles:
  Data:             single            8.00MiB
  Metadata:         DUP             264.00MiB
  System:           DUP              12.00MiB
SSD detected:       no
Incompat features:  extref, skinny-metadata
Number of devices:  1
Devices:
   ID        SIZE  PATH
    1     5.00GiB  /dev/loop0



In [53]:
sudo file -s /dev/loop0

/dev/loop0: BTRFS Filesystem sectorsize 4096, nodesize 16384, leafsize 16384, UUID=f21bfaef-88ef-40ac-b5f4- 45e2d84f7a2, 114688/5368709120 bytes used, 1 devices


In [54]:
sudo btrfs filesystem show

Label: none  uuid: f21bfaef-88ef-40ac-b5f4-045e2d84f7a2
	Total devices 1 FS bytes used 112.00KiB
	devid    1 size 5.00GiB used 536.00MiB path /dev/loop0



In [55]:
mkdir ../btrfs_c1

In [56]:
sudo mount /dev/loop0 ../btrfs_c1

In [57]:
cat /proc/mounts | grep btrfs

/dev/loop0 /home/vagrant/rdf/ais/btrfs_c1 btrfs rw,relatime,space_cache,subvolid=5,subvol=/ 0 0


In [58]:
sudo btrfs subvolume show ../btrfs_c1

/home/vagrant/rdf/ais/btrfs_c1 is toplevel subvolume


In [59]:
sudo btrfs filesystem df ../btrfs_c1

Data, single: total=8.00MiB, used=64.00KiB
System, DUP: total=8.00MiB, used=16.00KiB
Metadata, DUP: total=256.00MiB, used=112.00KiB
GlobalReserve, single: total=16.00MiB, used=0.00B


In [60]:
sudo lxc-create --bdev btrfs --lxcpath=$(pwd)/../btrfs_c1 --name btrfs_container --template ubuntu

Checking cache download in /var/cache/lxc/xenial/rootfs-amd64 ... 
Copy /var/cache/lxc/xenial/rootfs-amd64 to /home/vagrant/rdf/ais/hands-on-lxc/../btrfs_c1/btrfs_container/rootfs ... 
Copying rootfs to /home/vagrant/rdf/ais/hands-on-lxc/../btrfs_c1/btrfs_container/rootfs ...
Generating locales (this might take a while)...
  de_DE.UTF-8... done
Generation complete.
debconf: kann Oberfläche nicht initialisieren: Dialog
debconf: (Die Dialog-Oberfläche funktioniert nicht auf einem Dumb-Terminal, einem Emacs-Shellbuffer oder ohne ein steuerndes Terminal.)
debconf: greife zurück auf die Oberfläche: Readline
debconf: kann Oberfläche nicht initialisieren: Readline
debconf: (Can't locate Term/ReadLine.pm in @INC (you may need to install the Term::ReadLine module) (@INC contains: /etc/perl /usr/local/lib/x86_64-linux-gnu/perl/5.22.1 /usr/local/share/perl/5.22.1 /usr/lib/x86_64-linux-gnu/perl5/5.22 /usr/share/perl5 /usr/lib/x86_64-linux-gnu/perl/5.22 /usr/share/perl/5.22 /usr/local/lib/site_perl

In [61]:
sudo lxc-ls -f --lxcpath=$(pwd)/../btrfs_c1

NAME            STATE   AUTOSTART GROUPS IPV4 IPV6 
btrfs_container STOPPED 0         -      -    -    


In [62]:
sudo lxc-config lxc.lxcpath

/var/lib/lxc


In [63]:
echo "lxc.lxcpath = $(pwd)/../btrfs_c1" | sudo tee -a /etc/lxc/lxc.conf

lxc.lxcpath = /home/vagrant/rdf/ais/hands-on-lxc/../btrfs_c1


In [64]:
sudo lxc-config lxc.lxcpath

/home/vagrant/rdf/ais/hands-on-lxc/../btrfs_c1


In [65]:
sudo lxc-ls --fancy

NAME            STATE   AUTOSTART GROUPS IPV4 IPV6 
btrfs_container STOPPED 0         -      -    -    


In [66]:
sudo ls -la ../btrfs_c1/btrfs_container

insgesamt 20
drwxrwx--- 1 root root  24 Feb 17 18:00 .
drwxr-xr-x 1 root root  30 Feb 17 18:00 ..
-rw-r--r-- 1 root root 806 Feb 17 18:00 config
drwxr-xr-x 1 root root 132 Feb 16 17:28 rootfs


In [67]:
sudo btrfs subvolume list ../btrfs_c1

ID 257 gen 17 top level 5 path btrfs_container/rootfs


### Snapshot Btrfs

In [68]:
sudo lxc-copy -s -n btrfs_container -N btrfs_cow_clone

In [69]:
sudo lxc-ls -f

NAME            STATE   AUTOSTART GROUPS IPV4 IPV6 
btrfs_container STOPPED 0         -      -    -    
btrfs_cow_clone STOPPED 0         -      -    -    


In [70]:
sudo btrfs subvolume list ../btrfs_c1

ID 257 gen 18 top level 5 path btrfs_container/rootfs
ID 259 gen 18 top level 5 path btrfs_cow_clone/rootfs


In [71]:
sudo ls -la ../btrfs_c1

insgesamt 20
drwxr-xr-x 1 root    root      60 Feb 17 18:00 .
drwxrwxr-x 5 vagrant vagrant 4096 Feb 17 18:00 ..
drwxrwx--- 1 root    root      24 Feb 17 18:00 btrfs_container
drwxrwx--- 1 root    root      24 Feb 17 18:00 btrfs_cow_clone


In [72]:
sudo grep -vi ^# ../btrfs_c1/btrfs_container/config | grep lxc

lxc.include = /usr/share/lxc/config/ubuntu.common.conf
lxc.rootfs.path = btrfs:/home/vagrant/rdf/ais/hands-on-lxc/../btrfs_c1/btrfs_container/rootfs
lxc.uts.name = btrfs_container
lxc.arch = amd64
lxc.net.0.type = veth
lxc.net.0.link = lxcbr0
lxc.net.0.flags = up
lxc.net.0.hwaddr = 00:16:3e:15:56:4d


In [73]:
sudo lxc-start -n btrfs_cow_clone

In [74]:
sudo lxc-start -n btrfs_container

In [75]:
sudo lxc-ls -f

NAME            STATE   AUTOSTART GROUPS IPV4 IPV6 
btrfs_container RUNNING 0         -      -    -    
btrfs_cow_clone RUNNING 0         -      -    -    


### Aufräumen

In [76]:
sudo lxc-stop -n btrfs_cow_clone
sudo lxc-stop -n btrfs_container

In [77]:
sudo umount ../btrfs_c1

umount: /home/vagrant/rdf/ais/btrfs_c1: target is busy
        (In some cases useful info about processes that
         use the device is found by lsof(8) or fuser(1).)


: 32

In [78]:
echo "lxc.lxcpath = /var/lib/lxc" | sudo tee /etc/lxc/lxc.conf

lxc.lxcpath = /var/lib/lxc


## ZFS

In [82]:
apt policy zfsutils-linux

zfsutils-linux:
  Installiert:           0.6.5.6-0ubuntu18
  Installationskandidat: 0.6.5.6-0ubuntu18
  Versionstabelle:
 *** 0.6.5.6-0ubuntu18 500
        500 http://us.archive.ubuntu.com/ubuntu xenial-updates/main amd64 Packages
        100 /var/lib/dpkg/status
     0.6.5.6-0ubuntu8 500
        500 http://us.archive.ubuntu.com/ubuntu xenial/universe amd64 Packages


In [83]:
sudo lsmod | grep zfs
# modeprobe zfs

zfs                  2813952  3
zunicode              331776  1 zfs
zcommon                57344  1 zfs
znvpair                90112  2 zfs,zcommon
spl                   102400  3 zfs,zcommon,znvpair
zavl                   16384  1 zfs


In [3]:
sudo sgdisk -d 9 /dev/sdb 

The operation has completed successfully.


In [91]:
sudo partprobe

Error: Der/die Partition(en) 1, 9 auf /dev/sdb wurden geschrieben, aber es war nicht möglich, den Kernel über die Änderungen zu informieren, weil sie wahrscheinlich in Benutzung sind. Daher werden die alten Partitionen noch benutzt. Sie sollten nun den Rechner neu starten, bevor Sie weitere Änderungen vornehmen.


: 1

In [6]:
sudo sgdisk -p /dev/sdb

Disk /dev/sdb: 1048576000 sectors, 500.0 GiB
Logical sector size: 512 bytes
Disk identifier (GUID): 80A37054-FA86-BF41-930E-3F59303EC919
Partition table holds up to 128 entries
First usable sector is 34, last usable sector is 1048575966
Partitions will be aligned on 2048-sector boundaries
Total free space is 4029 sectors (2.0 MiB)

Number  Start (sector)    End (sector)  Size       Code  Name
   1            2048      1048557567   500.0 GiB   BF01  zfs-af1510068f8b7a18
   9      1048557568      1048573951   8.0 MiB     BF07  


In [5]:
sudo zpool create -f lxc sdb

In [6]:
sudo sgdisk -p /dev/sdb

Disk /dev/sdb: 1048576000 sectors, 500.0 GiB
Logical sector size: 512 bytes
Disk identifier (GUID): 80A37054-FA86-BF41-930E-3F59303EC919
Partition table holds up to 128 entries
First usable sector is 34, last usable sector is 1048575966
Partitions will be aligned on 2048-sector boundaries
Total free space is 4029 sectors (2.0 MiB)

Number  Start (sector)    End (sector)  Size       Code  Name
   1            2048      1048557567   500.0 GiB   BF01  zfs-af1510068f8b7a18
   9      1048557568      1048573951   8.0 MiB     BF07  


In [7]:
sudo zpool list

NAME   SIZE  ALLOC   FREE  EXPANDSZ   FRAG    CAP  DEDUP  HEALTH  ALTROOT
lxc    496G    64K   496G         -     0%     0%  1.00x  ONLINE  -


In [8]:
sudo zpool status

  pool: lxc
 state: ONLINE
  scan: none requested
config:

	NAME        STATE     READ WRITE CKSUM
	lxc         ONLINE       0     0     0
	  sdb       ONLINE       0     0     0

errors: No known data errors


In [9]:
sudo zfs list

NAME   USED  AVAIL  REFER  MOUNTPOINT
lxc     55K   480G    19K  /lxc


In [10]:
df -h | grep -w lxc

lxc                            481G       0  481G    0% /lxc


### Erstellen eines LXC Containers

In [12]:
sudo lxc-create --bdev zfs --lxcpath=/lxc --name zfs_container --template ubuntu

Checking cache download in /var/cache/lxc/xenial/rootfs-amd64 ... 
Copy /var/cache/lxc/xenial/rootfs-amd64 to /usr/lib/x86_64-linux-gnu/lxc ... 
Copying rootfs to /usr/lib/x86_64-linux-gnu/lxc ...
Generating locales (this might take a while)...
  de_DE.UTF-8... done
Generation complete.
debconf: kann Oberfläche nicht initialisieren: Dialog
debconf: (Die Dialog-Oberfläche funktioniert nicht auf einem Dumb-Terminal, einem Emacs-Shellbuffer oder ohne ein steuerndes Terminal.)
debconf: greife zurück auf die Oberfläche: Readline
debconf: kann Oberfläche nicht initialisieren: Readline
debconf: (Can't locate Term/ReadLine.pm in @INC (you may need to install the Term::ReadLine module) (@INC contains: /etc/perl /usr/local/lib/x86_64-linux-gnu/perl/5.22.1 /usr/local/share/perl/5.22.1 /usr/lib/x86_64-linux-gnu/perl5/5.22 /usr/share/perl5 /usr/lib/x86_64-linux-gnu/perl/5.22 /usr/share/perl/5.22 /usr/local/lib/site_perl /usr/lib/x86_64-linux-gnu/perl-base .) at /usr/share/perl5/Debconf/FrontEnd/Rea

In [13]:
sudo lxc-ls --lxcpath=/lxc -f

NAME          STATE   AUTOSTART GROUPS IPV4 IPV6 
zfs_container STOPPED 0         -      -    -    


In [14]:
ls -la /lxc

insgesamt 5
drwxr-xr-x  3 root root    3 Feb 17 18:20 .
drwxr-xr-x 25 root root 4096 Feb 17 18:16 ..
drwxrwx---  3 root root    4 Feb 17 18:20 zfs_container


### Snapshots ZFS

In [16]:
sudo lxc-copy --lxcpath=/lxc -s -n zfs_container -N zfs_cow_clone

In [17]:
sudo lxc-ls --lxcpath=/lxc -f

NAME          STATE   AUTOSTART GROUPS IPV4 IPV6 
zfs_container STOPPED 0         -      -    -    
zfs_cow_clone STOPPED 0         -      -    -    


In [14]:
ls -la /lxc

insgesamt 5
drwxr-xr-x  3 root root    3 Feb 17 18:20 .
drwxr-xr-x 25 root root 4096 Feb 17 18:16 ..
drwxrwx---  3 root root    4 Feb 17 18:20 zfs_container


In [18]:
sudo lxc-start --lxcpath=/lxc --name zfs_container
sudo lxc-start --lxcpath=/lxc --name zfs_cow_clone
sleep 8
sudo lxc-ls --lxcpath=/lxc -f

NAME          STATE   AUTOSTART GROUPS IPV4 IPV6 
zfs_container RUNNING 0         -      -    -    
zfs_cow_clone RUNNING 0         -      -    -    


In [19]:
sudo lxc-ls --lxcpath=/lxc -f

NAME          STATE   AUTOSTART GROUPS IPV4      IPV6 
zfs_container RUNNING 0         -      10.0.3.84 -    
zfs_cow_clone RUNNING 0         -      10.0.3.66 -    


In [20]:
sudo lxc-stop --lxcpath=/lxc --name zfs_cow_clone
sudo lxc-stop --lxcpath=/lxc --name zfs_container

In [21]:
sudo lxc-destroy --lxcpath=/lxc --name zfs_cow_clone
sudo lxc-destroy --lxcpath=/lxc --name zfs_container

Destroyed container zfs_cow_clone
Destroyed container zfs_container


In [24]:
ls -la /lxc

insgesamt 5
drwxr-xr-x  2 root root    2 Feb 17 18:25 .
drwxr-xr-x 25 root root 4096 Feb 17 18:16 ..


In [25]:
sudo zpool destroy lxc

In [26]:
sudo zpool list

no pools available


## Autostart von LXC Containern

In [40]:
sudo lxc-create --name autostart_container --template ubuntu

Checking cache download in /var/cache/lxc/xenial/rootfs-amd64 ... 
Copy /var/cache/lxc/xenial/rootfs-amd64 to /var/lib/lxc/autostart_container/rootfs ... 
Copying rootfs to /var/lib/lxc/autostart_container/rootfs ...
Generating locales (this might take a while)...
  de_DE.UTF-8... done
Generation complete.
debconf: kann Oberfläche nicht initialisieren: Dialog
debconf: (Die Dialog-Oberfläche funktioniert nicht auf einem Dumb-Terminal, einem Emacs-Shellbuffer oder ohne ein steuerndes Terminal.)
debconf: greife zurück auf die Oberfläche: Readline
debconf: kann Oberfläche nicht initialisieren: Readline
debconf: (Can't locate Term/ReadLine.pm in @INC (you may need to install the Term::ReadLine module) (@INC contains: /etc/perl /usr/local/lib/x86_64-linux-gnu/perl/5.22.1 /usr/local/share/perl/5.22.1 /usr/lib/x86_64-linux-gnu/perl5/5.22 /usr/share/perl5 /usr/lib/x86_64-linux-gnu/perl/5.22 /usr/share/perl/5.22 /usr/local/lib/site_perl /usr/lib/x86_64-linux-gnu/perl-base .) at /usr/share/perl5/

In [41]:
echo "lxc.start.auto = 1" | sudo tee -a /var/lib/lxc/autostart_container/config

lxc.start.auto = 1


In [42]:
sudo lxc-autostart --list

autostart_container 0


In [43]:
sudo lxc-autostart --all

In [48]:
sudo lxc-ls --fancy

NAME                STATE   AUTOSTART GROUPS IPV4 IPV6 
autostart_container STOPPED 1         -      -    -    
container_2_copy    STOPPED 0         -      -    -    
container_2_hard    STOPPED 0         -      -    -    
lvm_container       STOPPED 0         -      -    -    
lvm_container_2     STOPPED 0         -      -    -    


In [45]:
sudo lxc-stop --name autostart_container

In [49]:
echo "lxc.start.delay = 5" | sudo tee -a /var/lib/lxc/autostart_container/config

lxc.start.delay = 5


In [50]:
echo "lxc.group = high_priority" | sudo tee -a /var/lib/lxc/autostart_container/config

lxc.group = high_priority


In [51]:
sudo lxc-autostart --list

In [52]:
sudo lxc-autostart --list --group high_priority

autostart_container 5


In [53]:
sudo lxc-autostart --group high_priority

In [55]:
sudo lxc-ls --fancy

NAME                STATE   AUTOSTART GROUPS        IPV4      IPV6 
autostart_container RUNNING 1         high_priority 10.0.3.30 -    
container_2_copy    STOPPED 0         -             -         -    
container_2_hard    STOPPED 0         -             -         -    
lvm_container       STOPPED 0         -             -         -    
lvm_container_2     STOPPED 0         -             -         -    


In [57]:
sudo lxc-destroy --name autostart_container --force

Destroyed container autostart_container


## LXC hooks

| Option           | 	Description  |
|------------------|-----------------|
| lxc.hook.pre-start|	A hook to be run in the host namespace before the container ttys, consoles, or mounts are loaded |
| lxc.hook.pre-mount|	A hook to be run in the container's filesystem namespace, but before the rootfs has been set up |
| lxc.hook.mount| 	A hook to be run in the container after mounting has been done, but before the pivot_root |
| lxc.hook.autodev| 	A hook to be run in the container after mounting has been done and after any mount hooks have run, but before the pivot_root |
| lxc.hook.start| 	A hook to be run in the container right before executing the container's init |
| lxc.hook.stop| 	A hook to be run in the host's namespace after the container has been shut down |
|lxc.hook.post-stop |	A hook to be run in the host's namespace after the container has been shut down |
|lxc.hook.clone | 	A hook to be run when the container is cloned |
|lxc.hook.destroy | 	A hook to be run when the container is destroyed |



In [58]:
sudo lxc-create --name hooks_container --template ubuntu

Checking cache download in /var/cache/lxc/xenial/rootfs-amd64 ... 
Copy /var/cache/lxc/xenial/rootfs-amd64 to /var/lib/lxc/hooks_container/rootfs ... 
Copying rootfs to /var/lib/lxc/hooks_container/rootfs ...
Generating locales (this might take a while)...
  de_DE.UTF-8... done
Generation complete.
debconf: kann Oberfläche nicht initialisieren: Dialog
debconf: (Die Dialog-Oberfläche funktioniert nicht auf einem Dumb-Terminal, einem Emacs-Shellbuffer oder ohne ein steuerndes Terminal.)
debconf: greife zurück auf die Oberfläche: Readline
debconf: kann Oberfläche nicht initialisieren: Readline
debconf: (Can't locate Term/ReadLine.pm in @INC (you may need to install the Term::ReadLine module) (@INC contains: /etc/perl /usr/local/lib/x86_64-linux-gnu/perl/5.22.1 /usr/local/share/perl/5.22.1 /usr/lib/x86_64-linux-gnu/perl5/5.22 /usr/share/perl5 /usr/lib/x86_64-linux-gnu/perl/5.22 /usr/share/perl/5.22 /usr/local/lib/site_perl /usr/lib/x86_64-linux-gnu/perl-base .) at /usr/share/perl5/Debconf/

In [59]:
echo "lxc.hook.pre-start = /var/lib/lxc/hooks_container/pre_start.sh" | sudo tee -a /var/lib/lxc/hooks_container/config


lxc.hook.pre-start = /var/lib/lxc/hooks_container/pre_start.sh


In [61]:
cat pre_start.sh

#!/bin/bash
LOG_FILE=/tmp/container.log
echo "Container name: $LXC_NAME" | tee -a $LOG_FILE
echo "Container mounted rootfs: $LXC_ROOTFS_MOUNT" | tee -a
$LOG_FILE
echo "Container config file $LXC_CONFIG_FILE" | tee -a $LOG_FILE
echo "Container rootfs: $LXC_ROOTFS_PATH" | tee -a $LOG_FILE


In [62]:
sudo cp pre_start.sh /var/lib/lxc/hooks_container/pre_start.sh

In [64]:
sudo chmod u+x /var/lib/lxc/hooks_container/pre_start.sh

In [65]:
sudo  lxc-start --name hooks_container

In [66]:
sudo lxc-ls --fancy

NAME             STATE   AUTOSTART GROUPS IPV4      IPV6 
container_2_copy STOPPED 0         -      -         -    
container_2_hard STOPPED 0         -      -         -    
hooks_container  RUNNING 0         -      10.0.3.65 -    
lvm_container    STOPPED 0         -      -         -    
lvm_container_2  STOPPED 0         -      -         -    


In [67]:
cat /tmp/container.log

Container name: 
Container config file 
Container rootfs: 
Container name: hooks_container
Container config file /var/lib/lxc/hooks_container/config
Container rootfs: dir:/var/lib/lxc/hooks_container/rootfs


## Einhängen von Host-Dateien/Verzeichnissen

In [82]:
mkdir /tmp/export_to_container
hostname -f >> /tmp/export_to_container/file
sudo lxc-create --name mount_container --template ubuntu

mkdir: das Verzeichnis »/tmp/export_to_container“ kann nicht angelegt werden: Die Datei existiert bereits
rdf-dev.vm
Container already exists


: 1

In [69]:
echo "lxc.mount.entry = /tmp/export_to_container/ /var/lib/lxc/mount_container/rootfs/mnt none ro,bind 0 0" | sudo tee -a /var/lib/lxc/mount_container/config


lxc.mount.entry = /tmp/export_to_container/ /var/lib/lxc/mount_container/rootfs/mnt none ro,bind 0 0


In [85]:
sudo lxc-start --name mount_container

In [86]:
sudo lxc-ls --fancy

NAME             STATE   AUTOSTART GROUPS IPV4       IPV6 
container_2_copy STOPPED 0         -      -          -    
container_2_hard STOPPED 0         -      -          -    
hooks_container  RUNNING 0         -      10.0.3.65  -    
lvm_container    STOPPED 0         -      -          -    
lvm_container_2  STOPPED 0         -      -          -    
mount_container  RUNNING 0         -      10.0.3.166 -    


In [91]:
ssh-keygen -R 10.0.3.166
ssh-keyscan 10.0.3.166 >> ~/.ssh/known_hosts

# Host 10.0.3.166 found: line 1
/home/vagrant/.ssh/known_hosts updated.
Original contents retained as /home/vagrant/.ssh/known_hosts.old
# 10.0.3.166:22 SSH-2.0-OpenSSH_7.2p2 Ubuntu-4ubuntu2.4
# 10.0.3.166:22 SSH-2.0-OpenSSH_7.2p2 Ubuntu-4ubuntu2.4
# 10.0.3.166:22 SSH-2.0-OpenSSH_7.2p2 Ubuntu-4ubuntu2.4


In [92]:
sshpass -p ubuntu ssh ubuntu@10.0.3.166 cat /mnt/file

rdf-dev.vm


In [93]:
sudo lxc-info -n mount_container

Name:           mount_container
State:          RUNNING
PID:            17282
IP:             10.0.3.166
CPU use:        0.56 seconds
BlkIO use:      40.00 KiB
Memory use:     12.52 MiB
KMem use:       0 bytes
Link:           veth2QYQL4
 TX bytes:      29.37 KiB
 RX bytes:      29.36 KiB
 Total bytes:   58.73 KiB


In [94]:
sudo ls -la /proc/17282/root/run

insgesamt 28
drwxr-xr-x 12 root root  440 Feb 17 20:34 .
drwxr-xr-x 21 root root 4096 Feb 16 17:28 ..
-rw-------  1 root root    0 Feb 17 20:31 agetty.reload
-rw-r--r--  1 root root    3 Feb 17 20:31 crond.pid
----------  1 root root    0 Feb 17 20:31 crond.reboot
-rw-r--r--  1 root root    4 Feb 17 20:31 dhclient.eth0.pid
lrwxrwxrwx  1 root root   25 Feb 17 20:31 initctl -> /run/systemd/initctl/fifo
drwxrwxrwt  3 root root   60 Feb 17 20:31 lock
drwxr-xr-x  3 root root   60 Feb 17 20:31 log
-rw-r--r--  1 root root  210 Feb 17 20:34 motd.dynamic
drwxr-xr-x  2 root root   60 Feb 17 20:31 mount
drwxr-xr-x  2 root root  120 Feb 17 20:31 network
drwxr-xr-x  3 root root  100 Feb 17 20:31 resolvconf
-rw-r--r--  1 root root    2 Feb 17 20:31 rsyslogd.pid
drwxr-xr-x  2 root root   40 Feb 17 20:31 sendsigs.omit.d
lrwxrwxrwx  1 root root    8 Feb 17 20:31 shm -> /dev/shm
drwxr-xr-x  2 root root   40 Feb 17 20:31 sshd
-rw-r--r--  1 root root    4 Feb 17 20:31 sshd.pid
drwx--x--x  3 root root   60

## Freezing LXC Container

In [95]:
sudo lxc-ls -f

NAME             STATE   AUTOSTART GROUPS IPV4       IPV6 
container_2_copy STOPPED 0         -      -          -    
container_2_hard STOPPED 0         -      -          -    
hooks_container  RUNNING 0         -      10.0.3.65  -    
lvm_container    STOPPED 0         -      -          -    
lvm_container_2  STOPPED 0         -      -          -    
mount_container  RUNNING 0         -      10.0.3.166 -    


In [96]:
cat /sys/fs/cgroup/freezer/lxc/mount_container/freezer.state

THAWED


In [97]:
sudo lxc-freeze -n mount_container

In [98]:
sudo lxc-ls -f

NAME             STATE   AUTOSTART GROUPS IPV4       IPV6 
container_2_copy STOPPED 0         -      -          -    
container_2_hard STOPPED 0         -      -          -    
hooks_container  RUNNING 0         -      10.0.3.65  -    
lvm_container    STOPPED 0         -      -          -    
lvm_container_2  STOPPED 0         -      -          -    
mount_container  FROZEN  0         -      10.0.3.166 -    


In [99]:
cat /sys/fs/cgroup/freezer/lxc/mount_container/freezer.state

FROZEN


In [102]:
sudo lxc-unfreeze --name mount_container

In [103]:
sudo lxc-ls -f

NAME             STATE   AUTOSTART GROUPS IPV4       IPV6 
container_2_copy STOPPED 0         -      -          -    
container_2_hard STOPPED 0         -      -          -    
hooks_container  RUNNING 0         -      10.0.3.65  -    
lvm_container    STOPPED 0         -      -          -    
lvm_container_2  STOPPED 0         -      -          -    
mount_container  RUNNING 0         -      10.0.3.166 -    


## Limit setzen

In [105]:
sudo lxc-cgroup -n mount_container memory.limit_in_bytes 536870912

In [106]:
sudo cat /sys/fs/cgroup/memory/lxc/mount_container/memory.limit_in_bytes

536870912


In [107]:
sudo lxc-cgroup -n mount_container memory.limit_in_bytes 268435456

In [108]:
sudo cat /sys/fs/cgroup/memory/lxc/mount_container/memory.limit_in_bytes

268435456


In [109]:
sshpass -p ubuntu ssh ubuntu@10.0.3.166 free -m

              gesamt       benutzt     frei      gemns.  Puffer/Cache verfügbar
Speicher:         256           5         242          58           8         242
Auslagerungsspeicher:        2047          36        2011


In [110]:
sudo cat /sys/fs/cgroup/cpuset/lxc/mount_container/cpuset.cpus

0-1


In [111]:
cat /proc/cpuinfo | grep processor

processor	: 0
processor	: 1


In [112]:
sudo lxc-cgroup -n mount_container cpuset.cpus 0

In [113]:
sudo cat /sys/fs/cgroup/cpuset/lxc/mount_container/cpuset.cpus

0


In [114]:
sshpass -p ubuntu ssh ubuntu@10.0.3.166 cat /proc/cpuinfo | grep processor

processor	: 0


## Libvirt

In [118]:
ls /usr/lib/libvirt/libvirt_lxc

/usr/lib/libvirt/libvirt_lxc


In [119]:
sudo virsh uri

qemu:///system



In [120]:
export LIBVIRT_DEFAULT_URI=lxc:///

In [121]:
virsh uri

lxc:///



In [146]:
sudo virsh --connect lxc:/// list --all

 Id    Name                           Status
----------------------------------------------------
 -     libvirt_container1             Ausgeschaltet



In [127]:
sudo debootstrap --arch=amd64 --include="openssh-server vim" stable /root/container http://httpredir.debian.org/debian/

W: Cannot check Release signature; keyring file not available /usr/share/keyrings/debian-archive-keyring.gpg
I: Retrieving InRelease 
I: Failed to retrieve InRelease
I: Retrieving Release 
I: Retrieving Packages 
I: Validating Packages 
I: Resolving dependencies of required packages...
I: Resolving dependencies of base packages...
I: Found additional required dependencies: libaudit-common libaudit1 libbz2-1.0 libcap-ng0 libdb5.3 libdebconfclient0 libgcrypt20 libgpg-error0 liblz4-1 libncursesw5 libsemanage-common libsemanage1 libsystemd0 libudev1 libustr-1.0-1 
I: Found additional base dependencies: dmsetup gnupg-agent libapparmor1 libassuan0 libbsd0 libcap2 libcryptsetup4 libdevmapper1.02.1 libdns-export162 libedit2 libelf1 libfastjson4 libffi6 libgmp10 libgnutls30 libgpm2 libgssapi-krb5-2 libhogweed4 libidn11 libidn2-0 libip4tc0 libip6tc0 libiptc0 libisc-export160 libk5crypto3 libkeyutils1 libkrb5-3 libkrb5support0 libksba8 liblocale-gettext-perl liblognorm5 libmnl0 libncurses5 libnet

I: Retrieving libsemanage-common 2.6-2
I: Validating libsemanage-common 2.6-2
I: Retrieving libsemanage1 2.6-2
I: Validating libsemanage1 2.6-2
I: Retrieving libsepol1 2.6-2
I: Validating libsepol1 2.6-2
I: Retrieving libtasn1-6 4.10-1.1
I: Validating libtasn1-6 4.10-1.1
I: Retrieving libtext-charwidth-perl 0.04-7+b5
I: Validating libtext-charwidth-perl 0.04-7+b5
I: Retrieving libtext-iconv-perl 1.7-5+b4
I: Validating libtext-iconv-perl 1.7-5+b4
I: Retrieving libtext-wrapi18n-perl 0.06-7.1
I: Validating libtext-wrapi18n-perl 0.06-7.1
I: Retrieving libunistring0 0.9.6+really0.9.3-0.1
I: Validating libunistring0 0.9.6+really0.9.3-0.1
I: Retrieving logrotate 3.11.0-0.1
I: Validating logrotate 3.11.0-0.1
I: Retrieving lsb-base 9.20161125
I: Validating lsb-base 9.20161125
I: Retrieving dmsetup 2:1.02.137-2
I: Validating dmsetup 2:1.02.137-2
I: Retrieving libdevmapper1.02.1 2:1.02.137-2
I: Validating libdevmapper1.02.1 2:1.02.137-2
I: Retrieving liblz4-1 0.0~r131-2+b1
I: Validating liblz4-1 

I: Unpacking libbz2-1.0:amd64...
I: Unpacking libdebconfclient0:amd64...
I: Unpacking coreutils...
I: Unpacking dash...
I: Unpacking libdb5.3:amd64...
I: Unpacking debconf...
I: Unpacking debianutils...
I: Unpacking diffutils...
I: Unpacking dpkg...
I: Unpacking e2fslibs:amd64...
I: Unpacking e2fsprogs...
I: Unpacking libcomerr2:amd64...
I: Unpacking libss2:amd64...
I: Unpacking findutils...
I: Unpacking gcc-6-base:amd64...
I: Unpacking libgcc1:amd64...
I: Unpacking libc-bin...
I: Unpacking libc6:amd64...
I: Unpacking multiarch-support...
I: Unpacking grep...
I: Unpacking gzip...
I: Unpacking hostname...
I: Unpacking init-system-helpers...
I: Unpacking libcap-ng0:amd64...
I: Unpacking libgcrypt20:amd64...
I: Unpacking libgpg-error0:amd64...
I: Unpacking libselinux1:amd64...
I: Unpacking libsemanage-common...
I: Unpacking libsemanage1:amd64...
I: Unpacking libsepol1:amd64...
I: Unpacking lsb-base...
I: Unpacking liblz4-1:amd64...
I: Unpacking mawk...
I: Unpacking libncursesw5:amd64...
I

I: Configuring libffi6:amd64...
I: Configuring libtext-charwidth-perl...
I: Configuring libkeyutils1:amd64...
I: Configuring bsdmainutils...
I: Configuring libslang2:amd64...
I: Configuring cron...
I: Configuring libmnl0:amd64...
I: Configuring libassuan0:amd64...
I: Configuring logrotate...
I: Configuring rsyslog...
I: Configuring libip6tc0:amd64...
I: Configuring libwrap0:amd64...
I: Configuring netbase...
I: Configuring iputils-ping...
I: Configuring libedit2:amd64...
I: Configuring vim-tiny...
I: Configuring libapt-pkg5.0:amd64...
I: Configuring libapt-inst2.0:amd64...
I: Configuring libxapian30:amd64...
I: Configuring libidn2-0:amd64...
I: Configuring libnetfilter-conntrack3:amd64...
I: Configuring vim...
I: Configuring libpsl5:amd64...
I: Configuring libdns-export162...
I: Configuring libiptc0:amd64...
I: Configuring iproute2...
I: Configuring libkrb5support0:amd64...
I: Configuring isc-dhcp-client...
I: Configuring libhogweed4:amd64...
I: Configuring iptables...
I: Configuring l

In [145]:
sudo -H virsh --connect lxc:/// define libvirt_container1.xml

Domain libvirt_container1 von libvirt_container1.xml definiert



In [146]:
sudo virsh --connect lxc:/// list --all

 Id    Name                           Status
----------------------------------------------------
 -     libvirt_container1             Ausgeschaltet



In [148]:
sudo  virsh --connect lxc:/// start libvirt_container1 

Domain libvirt_container1 gestartet



In [149]:
sudo virsh --connect lxc:/// list --all

 Id    Name                           Status
----------------------------------------------------
 29647 libvirt_container1             laufend



In [133]:
sudo  lxc-create --name lxc-container --template ubuntu

Checking cache download in /var/cache/lxc/xenial/rootfs-amd64 ... 
Copy /var/cache/lxc/xenial/rootfs-amd64 to /var/lib/lxc/lxc-container/rootfs ... 
Copying rootfs to /var/lib/lxc/lxc-container/rootfs ...
Generating locales (this might take a while)...
  de_DE.UTF-8... done
Generation complete.
debconf: kann Oberfläche nicht initialisieren: Dialog
debconf: (Die Dialog-Oberfläche funktioniert nicht auf einem Dumb-Terminal, einem Emacs-Shellbuffer oder ohne ein steuerndes Terminal.)
debconf: greife zurück auf die Oberfläche: Readline
debconf: kann Oberfläche nicht initialisieren: Readline
debconf: (Can't locate Term/ReadLine.pm in @INC (you may need to install the Term::ReadLine module) (@INC contains: /etc/perl /usr/local/lib/x86_64-linux-gnu/perl/5.22.1 /usr/local/share/perl/5.22.1 /usr/lib/x86_64-linux-gnu/perl5/5.22 /usr/share/perl5 /usr/lib/x86_64-linux-gnu/perl/5.22 /usr/share/perl/5.22 /usr/local/lib/site_perl /usr/lib/x86_64-linux-gnu/perl-base .) at /usr/share/perl5/Debconf/Fron

In [142]:
sudo virsh domxml-from-native lxc-tools /var/lib/lxc/lxc-container/config | tee -a lxc-container.xml

Fehler: Ungültiges Argument: Nicht unterstützter Konfigurationstyp lxc-tools

